In [ ]:
!pip install transformers datasets evaluate seqeval


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=f63192b3681a697cad3b357a6d20e53d3bfd5520515ce2c50606167620c67c98
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.

In [ ]:
from transformers import AutoTokenizer, DataCollatorForTokenClassification, Trainer, TrainingArguments, AutoModelForTokenClassification
from datasets import Dataset
from sklearn.metrics import classification_report
import numpy as np
import torch


In [ ]:
import os
import zipfile

# Unzip datasets
dataset_zip_files = {
    "BC2GM": "/content/BC2GM-IOBES.zip",
    "BC4CHEMD": "/content/BC4CHEMD-IOBES.zip",
    "BC5CDR": "/content/BC5CDR-IOBES.zip",
    "JNLPBA": "/content/JNLPBA-IOBES.zip",
    "NCBI": "/content/NCBI-IOBES.zip"
}

all_datasets = {}
for task, zip_path in dataset_zip_files.items():
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(f"/content/{task}")
    train_file = f"/content/{task}/{task}-IOBES/train.tsv"
    test_file = f"/content/{task}/{task}-IOBES/test.tsv"

    def read_tsv(file_path):
        sentences, labels = [], []
        sentence, label = [], []
        with open(file_path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if line:
                    token, tag = line.split("\t")
                    sentence.append(token)
                    label.append(tag)
                else:
                    if sentence:
                        sentences.append(sentence)
                        labels.append(label)
                        sentence, label = [], []
        if sentence:
            sentences.append(sentence)
            labels.append(label)
        return {"sentences": sentences, "labels": labels}

    all_datasets[task] = {
        "train": read_tsv(train_file),
        "test": read_tsv(test_file),
    }


In [ ]:

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")

label_to_id = {}
id_to_label = {}
for task in all_datasets:
    unique_labels = set(tag for label_seq in all_datasets[task]["train"]["labels"] + all_datasets[task]["test"]["labels"] for tag in label_seq)
    label_to_id[task] = {label: idx for idx, label in enumerate(sorted(unique_labels))}
    id_to_label[task] = {idx: label for label, idx in label_to_id[task].items()}


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

In [ ]:
from datasets import Dataset

def preprocess_data(sentences, labels, label_to_id):
    tokenized_inputs = tokenizer(sentences, truncation=True, padding=True, is_split_into_words=True, max_length=128)
    tokenized_labels = []
    for i, label_seq in enumerate(labels):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        tokenized_labels.append([-100 if word_idx is None else label_to_id[label_seq[word_idx]] for word_idx in word_ids])
    tokenized_inputs["labels"] = tokenized_labels
    return tokenized_inputs

tokenized_datasets = {}
for task in all_datasets:
    tokenized_train = preprocess_data(
        all_datasets[task]["train"]["sentences"],
        all_datasets[task]["train"]["labels"],
        label_to_id[task]
    )
    tokenized_test = preprocess_data(
        all_datasets[task]["test"]["sentences"],
        all_datasets[task]["test"]["labels"],
        label_to_id[task]
    )
    tokenized_datasets[task] = {
        "train": Dataset.from_dict(tokenized_train),
        "test": Dataset.from_dict(tokenized_test),
    }


In [ ]:
class MultiTaskNER(torch.nn.Module):
    def __init__(self, num_labels_dict):
        super(MultiTaskNER, self).__init__()
        self.models = torch.nn.ModuleDict({
            task: AutoModelForTokenClassification.from_pretrained(
                "dmis-lab/biobert-base-cased-v1.1",
                num_labels=num_labels
            )
            for task, num_labels in num_labels_dict.items()
        })

    def forward(self, input_ids, attention_mask, labels=None, task_name=None):
        return self.models[task_name](input_ids=input_ids, attention_mask=attention_mask, labels=labels)

num_labels_dict = {task: len(label_to_id[task]) for task in all_datasets}
multi_task_model = MultiTaskNER(num_labels_dict)


pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForTokenClassification were not initialized from the model checkp

In [ ]:
def compute_metrics(predictions, labels, id_to_label):
    predictions = np.argmax(predictions, axis=2)

    # Convert predictions and labels to human-readable format
    true_predictions = [
        id_to_label[p]
        for prediction, label in zip(predictions, labels)
        for p, l in zip(prediction, label) if l != -100
    ]
    true_labels = [
        id_to_label[l]
        for prediction, label in zip(predictions, labels)
        for p, l in zip(prediction, label) if l != -100
    ]

    # Generate the classification report
    report = classification_report(
        true_labels,
        true_predictions,
        zero_division=0,
    )
    return {"classification_report": report}

for task in tokenized_datasets:
    training_args = TrainingArguments(
        output_dir=f"./results/{task}",
        evaluation_strategy="steps",
        save_strategy="steps",
        learning_rate=5e-5,
        per_device_train_batch_size=8,
        num_train_epochs=3,
        weight_decay=0.01,
        save_total_limit=2,
        load_best_model_at_end=True
    )

    trainer = Trainer(
        model=multi_task_model.models[task],
        args=training_args,
        train_dataset=tokenized_datasets[task]["train"],
        eval_dataset=tokenized_datasets[task]["test"],
        tokenizer=tokenizer,
        compute_metrics=lambda p: compute_metrics(p.predictions, p.label_ids, id_to_label[task]),
    )

    print(f"Training Task: {task}")
    trainer.train()
    print(f"Evaluation Results for {task}:")
    trainer.evaluate()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-8-f9f0b81be861>:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Training Task: BC2GM


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


...

Training Task: NCBI


Step,Training Loss,Validation Loss,Classification Report
500,0.134000,0.093527,precision recall f1-score support B-Disease 0.83 0.90 0.86 1217 E-Disease 0.87 0.96 0.91 1152 I-Disease 0.82 0.89 0.85 976 O 0.99 0.99 0.99 28903 S-Disease 0.93 0.87 0.90 1123 accuracy 0.97 33371 macro avg 0.89 0.92 0.90 33371 weighted avg 0.98 0.97 0.98 33371
1000,0.044100,0.095639,precision recall f1-score support B-Disease 0.87 0.87 0.87 1217 E-Disease 0.91 0.94 0.92 1152 I-Disease 0.86 0.87 0.86 976 O 0.99 0.99 0.99 28903 S-Disease 0.88 0.91 0.90 1123 accuracy 0.98 33371 macro avg 0.90 0.92 0.91 33371 weighted avg 0.98 0.98 0.98 33371
1500,0.021800,0.109044,precision recall f1-score support B-Disease 0.88 0.90 0.89 1217 E-Disease 0.90 0.93 0.91 1152 I-Disease 0.84 0.91 0.87 976 O 0.99 0.99 0.99 28903 S-Disease 0.88 0.91 0.89 1123 accuracy 0.98 33371 macro avg 0.90 0.92 0.91 33371 weighted avg 0.98 0.98 0.98 33371
2000,0.010600,0.110503,precision recall f1-score support B-Disease 0.88 0.90 0.89 1217 E-Disease 0.90 0.95 0.92 1152 I-Disease 0.85 0.90 0.87 976 O 0.99 0.99 0.99 28903 S-Disease 0.91 0.90 0.90 1123 accuracy 0.98 33371 macro avg 0.91 0.93 0.92 33371 weighted avg 0.98 0.98 0.98 33371


Trainer is attempting to log a value of "              precision    recall  f1-score   support

   B-Disease       0.83      0.90      0.86      1217
   E-Disease       0.87      0.96      0.91      1152
   I-Disease       0.82      0.89      0.85       976
           O       0.99      0.99      0.99     28903
   S-Disease       0.93      0.87      0.90      1123

    accuracy                           0.97     33371
   macro avg       0.89      0.92      0.90     33371
weighted avg       0.98      0.97      0.98     33371
" of type <class 'str'> for key "eval/classification_report" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "              precision    recall  f1-score   support

   B-Disease       0.87      0.87      0.87      1217
   E-Disease       0.91      0.94      0.92      1152
   I-Disease       0.86      0.87      0.86       976
           O       0.99      0.99      0.99

Evaluation Results for NCBI:


Trainer is attempting to log a value of "              precision    recall  f1-score   support

   B-Disease       0.83      0.90      0.86      1217
   E-Disease       0.87      0.96      0.91      1152
   I-Disease       0.82      0.89      0.85       976
           O       0.99      0.99      0.99     28903
   S-Disease       0.93      0.87      0.90      1123

    accuracy                           0.97     33371
   macro avg       0.89      0.92      0.90     33371
weighted avg       0.98      0.97      0.98     33371
" of type <class 'str'> for key "eval/classification_report" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
